# Tool 관련

### 입력 정의
- 간단한 경우: docstring으로 정의
- 복잡한 입력의 정의가 필요한 경우: pydantic model로 정의

### [Runtime](https://reference.langchain.com/python/langgraph/runtime/?_gl=1*16sz1dw*_gcl_au*NzA4NDA3MzguMTc1NTQyNDgwNw..*_ga*MTMzNzExOTc0Ny4xNzU1NDI0Nzgw*_ga_47WX3HKKY2*czE3NjI1Nzk3NzIkbzMwJGcwJHQxNzYyNTc5NzcyJGo2MCRsMCRoMA..) Information 접근
- State
- Context
- Store
- Stream Writer

**`ToolRuntime`**
    - Tool 내에서 접근

**`ModelRequest`**
    - Middleware에서 접근


### Tool 출력 제어
[`Command`](https://reference.langchain.com/python/langgraph/types/?_gl=1*1xa623w*_gcl_au*NzA4NDA3MzguMTc1NTQyNDgwNw..*_ga*MTMzNzExOTc0Ny4xNzU1NDI0Nzgw*_ga_47WX3HKKY2*czE3NjI1MDkzNzQkbzI1JGcxJHQxNzYyNTA5NDQ2JGo1OSRsMCRoMA..#langgraph.types.Command)

### 하위 에이전트 제어
[Reference](https://docs.langchain.com/oss/python/langchain/multi-agent#control-the-output-from-the-subagent)

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv(".env")
llm_large = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=1.0
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0
)

llm_small = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=1.0
)

In [ ]:
from typing import Annotated, cast
from langchain.tools import tool, ToolRuntime, InjectedToolCallId
from langgraph.types import Command
from langchain.messages import ToolMessage, HumanMessage, SystemMessage
from pydantic import BaseModel, Field

class TravelInfo(BaseModel):
    destination: str = Field(
        description="여행 목적지입니다."
    )
    information: str = Field(
        description="먹거리, 명소, 역사 등 여행지에 대한 정보입니다."
    )


@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """지금까지의 대화 내용을 요약한다."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"


@tool
def tool_1(query: str) -> str:
    """여행 계획의 개요를 작성하는 도구입니다.
    
    Args:
        query: str = 여행지와 여행 목적
    Return:
        content: str = 여행 계획 개요"""
    
    response = llm_small.invoke(f"다음의 여행지와 여행 목적에 맞도록 여행 계획 개요 항목을 작성해 주세요.\n{query}")

    return str(response.content)

@tool
def tool_2(
    query: str,
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """여행 계획의 개요를 작성하는 도구입니다.
    
    Args:
        query: str = 여행지와 여행 목적
    Return:
        Command: 여행 계획 개요"""
    
    
    response = llm_small.invoke(f"다음의 여행지와 여행 목적에 맞도록 여행 계획 개요 항목을 작성해 주세요.\n{query}")

    return Command(update={
        "messages": [
            ToolMessage(
                content=str(response.content),
                tool_call_id=tool_call_id,
                name="tool_2"
            )
        ]
    })


@tool
def tool_3(
    query: str,
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """여행지에 대한 정보를 수집하는 도구입니다.
    
    Args:
        query: str = 여행지
    Return:
        Command: 여행지 정보"""
    

    response = llm_small.invoke([
        SystemMessage(content="당신은 한국의 여행지 전문가입니다."),
        HumanMessage(content=f"{query}의 먹거리, 명소, 역사 등 정보를 알려주세요.")
    ])

    return Command(update={
        "destination": query,
        "messages": [
            ToolMessage(
                content=response.content,
                tool_call_id=tool_call_id,
                name="tool_3",
            )
        ]
    })

In [2]:
a = [1,2,3,4,5]
b = [a[0]]
print(b)

[1]


## Tool Error Handling

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """툴의 execution error를 커스텀하여 메시지를 리턴한다."""
    try:
        print(request)
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"],
            name=request.tool_call["name"]
        )

### Tool calling test

In [4]:
from langchain_core.messages import ToolCall

test_result = tool_3.invoke({
    "args": {"query": "제주도"},
    "name": "tool_3",
    "type": "tool_call",
    "id": "test_call_id_123"
})

print("Command 객체:", test_result)
print("\n업데이트 내용:")
print("- destination:", test_result.update.get("destination"))
print("- messages:", test_result.update.get("messages")[:100])

Command 객체: Command(update={'destination': '제주도', 'messages': [ToolMessage(content='안녕하세요! 제주도의 매력적인 먹거리, 아름다운 명소, 그리고 풍부한 역사에 대해 궁금하시군요. 제주도는 정말 다채로운 매력을 가진 곳이라, 알려드릴 이야기가 많답니다. 하나씩 자세히 소개해 드릴게요.\n\n## 제주도의 먹거리: 바다와 땅이 선사하는 풍미\n\n제주도는 신선한 해산물과 육지에서 나는 귀한 식재료 덕분에 맛있는 음식이 가득합니다.\n\n**1. 해산물:**\n*   **흑돼지:** 제주 하면 가장 먼저 떠오르는 음식이 아닐까요? 제주 흑돼지는 쫄깃한 식감과 풍부한 육즙이 일품입니다. 삼겹살, 목살 구이는 물론이고 돔베고기, 흑돼지 불고기 등 다양한 방식으로 즐길 수 있습니다.\n*   **갈치:** 제주 앞바다에서 잡히는 싱싱한 갈치는 제주 여행의 필수 코스입니다. 통통하게 살이 오른 갈치를 구워 먹거나, 조림, 국으로 끓여 먹으면 그 맛이 일품입니다.\n*   **고등어회:** 신선한 고등어회는 비린 맛 없이 고소하고 부드러운 맛을 자랑합니다. 묵은지와 함께 쌈 싸 먹으면 더욱 맛있습니다.\n*   **해산물:** 전복, 소라, 성게, 딱새우 등 다양한 제철 해산물을 맛볼 수 있습니다. 특히 전복죽, 성게미역국은 아침 식사로 든든하고 영양가도 풍부합니다.\n\n**2. 제주 전통 음식:**\n*   **몸국:** 돼지뼈를 우려낸 육수에 돼지 내장과 쌀뜨물을 넣고 끓인 제주 전통 향토 음식입니다. 걸쭉하고 진한 국물 맛이 특징입니다.\n*   **빙떡:** 메밀가루 반죽을 얇게 부쳐 팥소를 넣어 말아 먹는 음식입니다. 심플하지만 제주 사람들의 소울푸드입니다.\n*   **각종 나물:** 제주도는 해풍을 맞고 자란 신선한 나물이 많습니다. 톳, 미역, 다시마 등 해조류를 활용한 나물 반찬이 인기가 많습니다.\n\n**3. 제주 특산품 활용 음식:**\n*   **감귤/한라봉:** 제주를 대표하는 감

In [ ]:
from util.testhelper import test_tool_invoke

response = test_tool_invoke(tool_1, {"query": "경주 역사여행"}, print_result=True)


🔧 Tool Invoke 테스트
📛 Tool Name: tool_1
🔑 Call ID: call_ce39ae59

📋 Input Arguments:
   - query: 경주 역사여행

⏳ Invoking tool...

✅ Tool Execution Success

📤 Result:

ToolMessage 객체

{
  "__class__": "ToolMessage",
  "content": "## 경주 역사여행 여행 계획 개요\n\n**여행 기간:** (예시: 2박 3일)\n\n**여행 테마:** 천년 고도 경주의 신라 역사 탐방\n\n**여행 목적:**\n\n*   신라 시대의 찬란한 유적과 문화를 직접 체험하고 이해한다.\n*   역사적 사실과 현재의 모습을 비교하며 생생한 역사 학습의 기회를 갖는다.\n*   전통 문화와 자연이 어우러진 경주의 매력을 느끼고 휴식을 취한다.\n*   (개인적인 관심사에 따라 추가: 특정 시대/인물/사건 집중 탐구 등)\n\n---\n\n### **여행 계획 개요**\n\n**1. 사전 준비 및 정보 수집**\n\n*   **여행 일정 확정:** 방문할 주요 유적지 및 체험 프로그램 리스트업\n*   **이동 수단 계획:** 대중교통, 자가용, 렌터카 등 이동 방식 결정 및 예약 (필요시)\n*   **숙소 예약:** 여행 테마와 동선에 맞는 숙소 선정 및 예약 (한옥 숙소, 시내 호텔 등)\n*   **필수..." ,
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "tool",
  "name": "tool_1",
  "id": null,
  "tool_call_id": "call_ce39ae59",
  "artifact": null,
  "status": "success"
}




## Agent에서 실행 테스트

In [9]:
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage

class State(AgentState):
    destination: str


agent = create_agent(
    model=llm,
    tools=[tool_2, tool_3],
    state_schema=State,
    system_prompt="당신은 한국의 여행사 직원입니다. 도구를 활용하여 고객의 여행 계획을 수립하세요." \
    "계획 수립에 앞서 먼저 여행지에 대한 조사를 수행하세요.",
    middleware=[handle_tool_errors]
)

response = agent.invoke({"messages": [HumanMessage("제주도 여행 계획을 세워주세요. 부모님을 모시고 효도 관광을 하고 싶습니다.")]})

ToolCallRequest(tool_call={'name': 'tool_3', 'args': {'query': '제주도'}, 'id': 'd7cd14db-c3f3-4627-8171-0a369a9a51bc', 'type': 'tool_call'}, tool=StructuredTool(name='tool_3', description='여행지에 대한 정보를 수집하는 도구입니다.\n\nArgs:\n    query: str = 여행지\nReturn:\n    Command: 여행지 정보', args_schema=<class 'langchain_core.utils.pydantic.tool_3'>, func=<function tool_3 at 0x1142fea20>), state={'messages': [HumanMessage(content='제주도 여행 계획을 세워주세요. 부모님을 모시고 효도 관광을 하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='bda23022-75a4-40bf-a94e-729a3a0cea1f'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tool_3', 'arguments': '{"query": "\\uc81c\\uc8fc\\ub3c4"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--68b4a9d8-0b78-4fff-b683-0cff7d676f8e-0', tool_calls=[{'name': 'tool_3', 'args': {'quer

In [ ]:
from util.testhelper import print_messages

print_messages(response["messages"], max_content_length=100)


📨 총 4개의 메시지

[1] HUMANMESSAGE (human)

💬 Content:
제주도 여행 계획을 세워주세요. 부모님을 모시고 효도 관광을 하고 싶습니다.

🔑 ID: bda23022-75a4-40bf-a94e-729a3a0cea1f
📛 Name: None
--------------------------------------------------------------------------------

[2] AIMESSAGE (ai)

💬 Content:


🔑 ID: lc_run--68b4a9d8-0b78-4fff-b683-0cff7d676f8e-0
📛 Name: None
🔧 Tool calls: 1
   - tool_3
     Args: {'query': '제주도'}
📊 Usage: {'input_tokens': 201, 'output_tokens': 124, 'total_tokens': 325, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 107}}
--------------------------------------------------------------------------------

[3] TOOLMESSAGE (tool)

💬 Content:
안녕하세요! 제주도 여행 전문가로서 제주도의 매력적인 먹거리, 아름다운 명소, 흥미로운 역사에 대한 모든 것을 상세하게 알려드릴게요.

---

### 🍊 제주도의 매력: 먹거리,...

🔑 ID: 8a56235f-01e0-41db-ac24-6226a44341a4
📛 Name: tool_3
--------------------------------------------------------------------------------

[4] AIMESSAGE (ai)

💬 Content:
부모님과 함께하는 제주도 효도 관광이시군요! 제주도는 자연 경관이 아름답고 먹거리가 풍부하며, 몸에 좋은 